In [1]:
from google.colab import files
uploaded = files.upload()

Saving MasterResearchFunction.py to MasterResearchFunction.py


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install japanize_matplotlib bottleneck tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 39.7 MB/s eta 0:00:00
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=6fc0e3adf7bd60aeb658006d55b0480c3a33ed2a170ac92aefd810aeeea78fe9
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib


In [4]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

In [5]:
#motion_data = mr.process_apple_watch_csv("datasets/kawano/20240517/MotionData_20240517_220252.csv")
motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/kawano/20240517/MotionData_20240517_220252.csv")

In [6]:
motion_data

,UnixTime,AccelerationX,AccelerationY,AccelerationZ,GyroX,GyroY,GyroZ,Timestamp,EuclideanNorm,SG_AccelerationX,SG_AccelerationY,SG_AccelerationZ,SG_EuclideanNorm,PowerSpectrum_AccelerationX,PowerSpectrum_AccelerationY,PowerSpectrum_AccelerationZ,PowerSpectrum_EuclideanNorm
0,1.715951e+09,-0.015587,-0.026884,-0.075159,-0.082577,0.017382,-0.062452,2024-05-17 22:02:53.006553856,0.081330,-0.047424,-0.036636,-0.080890,0.107378,1.884077e+07,2.952108e+06,6.177202e+06,3.609098e+08
1,1.715951e+09,-0.057833,-0.070985,-0.049163,-0.124807,0.074568,-0.096649,2024-05-17 22:02:53.025937152,0.103926,-0.067354,-0.050493,-0.048840,0.109263,1.378245e+07,3.512297e+06,3.197300e+06,1.690434e+08
2,1.715951e+09,-0.114945,-0.043187,-0.037461,-0.122649,0.123135,-0.118063,2024-05-17 22:02:53.047333120,0.128378,-0.080074,-0.059182,-0.022423,0.108995,8.792595e+06,2.000578e+06,1.752399e+06,9.178287e+07
3,1.715951e+09,-0.117376,-0.074545,-0.003240,-0.130185,0.158814,-0.106278,2024-05-17 22:02:53.066020096,0.139085,-0.085585,-0.062702,-0.001640,0.106573,4.970331e+06,9.580060e+05,7.704729e+05,4.283974e+07
4,1.715951e+09,-0.111981,-0.049324,0.017069,-0.031953,0.155571,-0.111159,2024-05-17 22:02:53.086338048,0.123548,-0.083887,-0.061054,0.013509,0.101998,2.008420e+06,3.200928e+05,2.310007e+05,1.524093e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174747,1.715954e+09,-0.009858,0.014656,0.025280,-0.003028,-0.048120,0.169169,2024-05-17 23:01:06.067841024,0.030839,-0.017278,0.003253,0.043156,0.054846,3.095552e+05,9.952137e+03,1.042533e+04,4.883268e+05
174748,1.715954e+09,-0.020856,-0.017834,0.018343,0.006847,-0.015403,0.157109,2024-05-17 23:01:06.088078848,0.033007,-0.013900,0.007567,0.035221,0.048093,2.008420e+06,3.200928e+05,2.310007e+05,1.524093e+07
174749,1.715954e+09,-0.012967,0.026964,0.003388,0.067122,0.007047,0.148085,2024-05-17 23:01:06.108173824,0.030111,-0.009779,0.011099,0.024058,0.039028,4.970331e+06,9.580060e+05,7.704729e+05,4.283974e+07
174750,1.715954e+09,-0.013979,0.021707,-0.010086,0.145702,0.058197,0.162458,2024-05-17 23:01:06.128171008,0.027718,-0.004914,0.013849,0.009669,0.027649,8.792595e+06,2.000578e+06,1.752399e+06,9.178287e+07


In [7]:
# check = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_check")
# circle = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_circle")
# cross = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_cross")
# tri = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_tri")
check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_check")
circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_circle")
cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_cross")
tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_tri")

In [ ]:
eye_data = mr.process_tobii_csv("datasets/kawano/20240517/kawano Recording002.csv")

In [ ]:
times_x, times_y, times_z = [], [], []
epsilon = 0.05
for train in circle:
    _, times = mr.plot_spring(motion_data['SG_AccelerationX'], train['SG_AccelerationX'], motion_data['Timestamp'], epsilon)
    times_x.append(times)
    _, times = mr.plot_spring(motion_data['SG_AccelerationY'], train['SG_AccelerationY'], motion_data['Timestamp'], epsilon)
    times_y.append(times)
    _, times = mr.plot_spring(motion_data['SG_AccelerationZ'], train['SG_AccelerationZ'], motion_data['Timestamp'], epsilon)
    times_z.append(times)

In [ ]:
times_x

In [11]:
times_y

[[8906   2024-05-17 22:05:51.025538048
  8905   2024-05-17 22:05:51.005543936
  8905   2024-05-17 22:05:51.005543936
  8905   2024-05-17 22:05:51.005543936
  8905   2024-05-17 22:05:51.005543936
                      ...             
  8853   2024-05-17 22:05:49.966375936
  8853   2024-05-17 22:05:49.966375936
  8853   2024-05-17 22:05:49.966375936
  8853   2024-05-17 22:05:49.966375936
  8853   2024-05-17 22:05:49.966375936
  Name: Timestamp, Length: 155, dtype: datetime64[ns],
  10890   2024-05-17 22:06:30.683845120
  10890   2024-05-17 22:06:30.683845120
  10890   2024-05-17 22:06:30.683845120
  10890   2024-05-17 22:06:30.683845120
  10889   2024-05-17 22:06:30.663404032
                       ...             
  10809   2024-05-17 22:06:29.064471040
  10809   2024-05-17 22:06:29.064471040
  10809   2024-05-17 22:06:29.064471040
  10809   2024-05-17 22:06:29.064471040
  10809   2024-05-17 22:06:29.064471040
  Name: Timestamp, Length: 163, dtype: datetime64[ns],
  16152   2024-05-17 

In [12]:
times_z

[[4678   2024-05-17 22:04:26.513086976
  4678   2024-05-17 22:04:26.513086976
  4678   2024-05-17 22:04:26.513086976
  4678   2024-05-17 22:04:26.513086976
  4678   2024-05-17 22:04:26.513086976
                      ...             
  4608   2024-05-17 22:04:25.113810176
  4608   2024-05-17 22:04:25.113810176
  4608   2024-05-17 22:04:25.113810176
  4608   2024-05-17 22:04:25.113810176
  4608   2024-05-17 22:04:25.113810176
  Name: Timestamp, Length: 153, dtype: datetime64[ns],
  9776   2024-05-17 22:06:08.415809280
  9776   2024-05-17 22:06:08.415809280
  9776   2024-05-17 22:06:08.415809280
  9776   2024-05-17 22:06:08.415809280
  9776   2024-05-17 22:06:08.415809280
                      ...             
  9646   2024-05-17 22:06:05.817324288
  9646   2024-05-17 22:06:05.817324288
  9646   2024-05-17 22:06:05.817324288
  9646   2024-05-17 22:06:05.817324288
  9646   2024-05-17 22:06:05.817324288
  Name: Timestamp, Length: 193, dtype: datetime64[ns],
  14269   2024-05-17 22:07:38.22

In [ ]:
S = 0.1
N = 50

In [ ]:
mask = motion_data['EuclideanNorm'] <= S
# 連続する True のブロックにグループ番号を割り当てる
mask_shifted = mask.ne(mask.shift(1))  # 状態が変わる箇所を検出
cumsum = mask_shifted.cumsum()  # 変化点で番号を増やしていく

In [ ]:
# N サンプル以上連続する True のグループを見つける
N = 50  # 例として 5 を連続カウントの閾値に設定
groups = mask.groupby(cumsum).transform('sum')  # グループごとの True の合計数を求める
to_remove = groups >= N  # N サンプル以上のグループを True とする

# 条件に一致する区間を削除
filtered_data = motion_data[~to_remove]

In [ ]:
filtered_data['Timestamp']

In [ ]:
pathes, times = mr.plot_spring(filtered_data['SG_AccelerationX'], circle1['AccelerationX'], filtered_data['Timestamp'], 0.1)